In [1]:
import time
import pickle
import argparse
import torch

from torch.utils.data import DataLoader
from esim.data import NLIDataset
from esim.model import ESIM
from esim.utils import correct_predictions
from sklearn import metrics

In [2]:
import esim

In [3]:
esim.__path__

['/home/rongz/anaconda3/lib/python3.6/site-packages/esim']

In [4]:
test_file='/home/rongz/ESIM/data/preprocessed/quora/test_data.pkl'
pretrained_file='/home/rongz/ESIM/data/checkpoints/quora/best.pth.tar'
batch_size=32

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
print(20 * "=", " Preparing for testing ", 20 * "=")

checkpoint = torch.load(pretrained_file)

# Retrieving model parameters from checkpoint.
vocab_size = checkpoint["model"]["_word_embedding.weight"].size(0)
embedding_dim = checkpoint["model"]['_word_embedding.weight'].size(1)
hidden_size = checkpoint["model"]["_projection.0.weight"].size(0)
num_classes = checkpoint["model"]["_classification.4.weight"].size(0)

====================  Preparing for testing  ====================


In [7]:
print("\t* Loading test data...")
with open(test_file, "rb") as pkl:
    test_data = NLIDataset(pickle.load(pkl))

test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

	* Loading test data...


In [8]:
print("\t* Building model...")
model = ESIM(vocab_size,
             embedding_dim,
             hidden_size,
             num_classes=num_classes,
             device=device).to(device)

model.load_state_dict(checkpoint["model"])

	* Building model...


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [9]:
model

ESIM(
  (_word_embedding): Embedding(128447, 300, padding_idx=0)
  (_rnn_dropout): RNNDropout(p=0.5)
  (_encoding): Seq2SeqEncoder(
    (_encoder): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (_attention): SoftmaxAttention()
  (_projection): Sequential(
    (0): Linear(in_features=2400, out_features=300, bias=True)
    (1): ReLU()
  )
  (_composition): Seq2SeqEncoder(
    (_encoder): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (_classification): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=2400, out_features=300, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.5)
    (4): Linear(in_features=300, out_features=2, bias=True)
  )
)

In [10]:
print(20 * "=", " Testing ESIM model on device: {} ".format(device), 20 * "=")
# Switch the model to eval mode.
model.eval()
device = model.device

time_start = time.time()
batch_time = 0.0
accuracy = 0.0

all_labels = []
all_out_classes = []

# Deactivate autograd for evaluation.
with torch.no_grad():
    for batch in test_loader:
        batch_start = time.time()

        # Move input and output data to the GPU if one is used.
        premises = batch["premise"].to(device)
        premises_lengths = batch["premise_length"].to(device)
        hypotheses = batch["hypothesis"].to(device)
        hypotheses_lengths = batch["hypothesis_length"].to(device)
        labels = batch["label"]
        all_labels.extend(labels.tolist())
        labels = labels.to(device)
        _, probs=model(premises,premises_lengths,hypotheses,hypotheses_lengths)
        break
        _, probs, prem_hyp_attn, hyp_prem_attn = model(premises,
                         premises_lengths,
                         hypotheses,
                         hypotheses_lengths)
        _, out_classes = probs.max(dim=1)
        all_out_classes.extend(out_classes.tolist())

        accuracy += correct_predictions(probs, labels)
        batch_time += time.time() - batch_start
        break
# batch_time /= len(test_loader)
# total_time = time.time() - time_start
# accuracy /= (len(test_loader.dataset))
# accuracy_score = metrics.accuracy_score(all_labels, all_out_classes)
# precision_score = metrics.precision_score(all_labels, all_out_classes)
# recall_score = metrics.recall_score(all_labels, all_out_classes)
# f1_score = metrics.f1_score(all_labels, all_out_classes)

====================  Testing ESIM model on device: cuda:0  ====================


/home/rongz/anaconda3/lib/python3.6/site-packages/esim/utils.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  sequences_lengths.new_tensor(torch.arange(0, len(sequences_lengths)))


ValueError: too many values to unpack (expected 2)